In [ ]:
import fxcmpy, socketio, engineio
print(fxcmpy.__version__)
print(engineio.__version__)
print(socketio.__version__)
TOKEN = "5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

In [ ]:
import datetime as dt
from icecream import ic 
import socketio, engineio


start = dt.datetime(2022,5,1)
end = dt.datetime(2022,5,3)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end
# data.to_csv("mar.csv")
data

In [303]:
kada = data.copy()

In [1]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])
# data.to_csv("feb.csv")

In [2]:
from icecream import ic
from datetime import datetime
import pandas_ta as pta
d1 = pd.read_csv("feb1.csv")
d2 = pd.read_csv("feb2.csv")
d3 = pd.read_csv("feb3.csv")
# d3 = pd.read_csv("second_jan.csv")
frames = [d1,d2,d3]
data = pd.concat(frames)
data.index = data.date.values

# # data = data.loc[(data["date"] > datetime.strptime("2022-01-05", "%Y-%m-%d"))]
data = data.drop(["date"], axis=1)
# data = data.iloc[40:3000] # 5831
data
# pd.DataFrame(index=data.date, data=data)


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
2022-02-01 00:00:00,1.12286,1.12282,1.12288,1.12277,1.12299,1.12294,1.12300,1.12289,118
2022-02-01 00:01:00,1.12281,1.12284,1.12290,1.12271,1.12294,1.12297,1.12303,1.12284,127
2022-02-01 00:02:00,1.12285,1.12291,1.12298,1.12285,1.12297,1.12304,1.12311,1.12297,36
2022-02-01 00:03:00,1.12292,1.12287,1.12292,1.12283,1.12304,1.12299,1.12304,1.12297,40
2022-02-01 00:04:00,1.12287,1.12292,1.12293,1.12285,1.12300,1.12305,1.12306,1.12298,53
...,...,...,...,...,...,...,...,...,...
2022-02-27 23:56:00,1.11811,1.11819,1.11821,1.11803,1.11824,1.11832,1.11833,1.11816,145
2022-02-27 23:57:00,1.11820,1.11813,1.11830,1.11807,1.11832,1.11829,1.11843,1.11823,235
2022-02-27 23:58:00,1.11812,1.11794,1.11812,1.11793,1.11829,1.11810,1.11829,1.11809,241
2022-02-27 23:59:00,1.11795,1.11797,1.11804,1.11793,1.11811,1.11811,1.11819,1.11808,162


In [612]:
# import ta
# from ta.trend import *
# from ta.momentum import *
from IPython.display import display


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)
data = kada

In [3]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import redis_client
importlib.reload(redis_client)
from redis_client import client as rs_client
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *
import copy



class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# data[:220]
# # for i in range(51,100):
# strategy.dataframe = data[:222]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]
# dd

# rs_client.set("ch", pickle.dumps(SMAStrategy))

In [4]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [5]:
class SMAStrategy2(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER" and \
                    last_sma_crossover.min_bid > last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and \
                    last_sma_crossover.max_bid < last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).iloc[1650:1700]

In [6]:
class SMAStrategy3Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""  
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        signal = self.entry_signal
        target = self.__class__.get_target_price(signal.entry_price, self.target, signal.entry_signal)
        stoploss = self.__class__.get_stoploss_price(signal.entry_price, self.stoploss, signal.entry_signal)
        df_last_date = df.tail(1).index[0]
        temp_df = df[:signal.Index][-10:]
        rows_min = min(temp_df.min_bid)
        rows_max = max(temp_df.max_bid)
        slp_price= 0.0002
        tgp_price = 0.0002
        new_df = df[signal.Index:df_last_date].iloc[1: , :][:100]
        exit_price = 0
        exit_time = None
        status = ""
        rsi_high_hit = 0
        rsi_low_hit = 0
        
        def last_prices_fetcher(price:float, storage:list):
            if len(storage) == 4:
                storage.append(price)
                storage.pop(0)
            else:
                storage.append(price)
            return storage
        
        def get_trend(price_list:list):
            avr = round(sum(price_list) / len(price_list), 5)
            # ic(price_list[-1] - avr)
            if price_list[-1] > avr:
                return "UP"
            elif price_list[-1] < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if signal.entry_signal == "BUY":
            last_row = None
            new_sl = min(signal.bidopen - slp_price, rows_min)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 70 and row.rsi < 70:
                    rsi_high_hit += 1
                if last_row.rsi < 30 and row.rsi > 30:
                    rsi_low_hit += 1
                
                # ic(last_row.rsi > 70 and row.rsi < 70)
                # ic(row.max_bid >= target)
                # ic(row)
                # ic(target)
                if new_tgp_row == None:
                    if row.max_bid >= signal.bidclose + tgp_price:
                        new_tgp_row = row  
                elif row.max_bid >= target:
                    target_hit = True
                    if row.rsi < 67:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.min_bid <= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose <= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi > 71:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi < 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                
                elif last_row.rsi > 70 and row.rsi < 70:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        elif signal.entry_signal == "SELL":
            last_row = None
            new_sl = max(rows_max, signal.bidopen + slp_price)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 30 and row.rsi < 30:
                    rsi_high_hit += 1
                if last_row.rsi < 70 and row.rsi > 70:
                    rsi_low_hit += 1
                if new_tgp_row == None:
                    if row.min_bid <= signal.bidclose - tgp_price:
                        new_tgp_row = row 
                elif row.min_bid <= target:
                    target_hit = True
                    if row.rsi > 33:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.max_bid >= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose >= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi < 29:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi > 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                elif last_row.rsi < 30 and row.rsi > 30:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        if all([exit_price, status, exit_time]):
            return exit_price, exit_time, status
        return 0,0,0

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value
            
    

    
class SMAExitLevel1(BaseStrategy):
    
    def get_next_entry_signal(self):
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        df = self.dataframe.copy()
        new_df = df[self.entry_signal.Index:]
        if new_df.empty:
            return None
        # this is temporary code for testing it need optimization
        for date_index in new_df.index:
            strategy.dataframe = df[:date_index]
            strategy.is_valid_dataframe()
            signal = strategy.get_signal()
            if signal:
                signal_df = pd.DataFrame(signal).iloc[0]
                if signal_df.name <= self.entry_signal.Index:
                    continue
                return signal
        return None
    
    def strategy(self):
        df = self.dataframe.copy()
        stoploss_price = self.__class__.get_stoploss_price(self.entry_signal.entry_price, self.stoploss, self.entry_signal.entry_signal)
        exit_strategy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        exit_strategy.dataframe = df
        exit_strategy.entry_signal = self.entry_signal
        exit_strategy.is_valid_dataframe()
        exit_signal = exit_strategy.get_signal()
        
        # return exit_signal
        next_entry_signal = self.get_next_entry_signal()
        # ic(next_entry_signal)
        if next_entry_signal:
            next_entry_signal = pd.DataFrame(next_entry_signal).iloc[0]
        else:
            next_entry_signal = pd.DataFrame()
        
        if exit_signal == (0,0,0):
            new_df = self.dataframe[self.entry_signal.Index:]
            counter = 0
            for frame in new_df.itertuples():
                if self.entry_signal.entry_signal == "BUY" and frame.bidclose <= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
                elif self.entry_signal.entry_signal == "SELL" and frame.bidclose >= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
        elif exit_signal == (0,0,0) and not next_entry_signal.empty:
            next_entry_time = next_entry_signal.name
            signal_entry_time = self.entry_signal.Index
            if next_entry_time > signal_entry_time:
                if self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "SELL":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose < -0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "BUY":
#                     for time being exit all entry
                    return next_entry_signal.bidclose, next_entry_time, "TGP"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "BUY":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose > 0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "SELL":
                # for time being exit and take new entry      
                    return next_entry_signal.bidclose, next_entry_time, "SLPR"
        return exit_signal


# SMAExitLevel1Copy = copy.deepcopy(SMAExitLevel1)
# d["entry_signal"] = {'2021-12-01 10:25:00': 'SELL'}
test_signal = next(pd.DataFrame(d).itertuples())
tragedy = SMAStrategy3Exit(target=0.0010, stoploss=0.0005)
kata = data[20:193]
print(kata.iloc[-1].name)
tragedy.dataframe = kata
tragedy.entry_signal = test_signal
tragedy.is_valid_dataframe()
dd= tragedy.get_signal()
ic(dd)
test_signal
# df_signal = next(pd.DataFrame(signal).itertuples())
#             strategy.entry_signal = next(pd.DataFrame(signal).itertuples())
#             # # # tragedy.is_valid_dataframe()
#             dd= strategy.get_signal()
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).head(50) .iloc[1:60]
# dd
# data.iloc[0].name

NameError: name 'd' is not defined

In [ ]:
ss = pd.DataFrame(sma_backtest.signal_data)
from strategy import DefaultExitStrategy
# bd = ss.itertuples()
# p = next(bd)
# ic(p)
# for i in bd:
#     ic(i)
#     break
# for test_signal in ss.itertuples():
#     if pd.Timestamp(test_signal.Index) == pd.Timestamp("2021-12-01 01:21:00"):
#         break
# test_signal

def run_exit_strategy(exit_strategy, signal_df, main_df):
    exit_details = []
    exit_strategy = exit_strategy(stoploss=0.0005, target=0.0008)
    exit_strategy.dataframe = main_df
    signal_data = signal_df.itertuples()
    next(signal_data)
    exit_details.append((0,0,0))
    for signal in signal_data:
        exit_strategy.entry_signal = signal
        if exit_strategy.is_valid_dataframe():
            # if signal == test_signal:
            #     ic(signal)
            #     ic(exit_strategy.get_signal())
            exit_details.append(exit_strategy.get_signal())

    signal_df["exit_price"] = [detail[0] for detail in exit_details]
    signal_df["exit_time"] = [detail[1] for detail in exit_details]
    signal_df["status"] = [detail[2] for detail in exit_details]
    return signal_df
dp = run_exit_strategy(SMAStrategy3Exit, ss, test_data)
dp

In [94]:
# dp.head(40)

In [7]:
import strategy
importlib.reload(strategy)
class SMAStrategy3Level1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    
    def create_signal(self, data, **kwargs):
        d_date = data.index[0]
        d = data.to_dict()
        d["entry_price"] = d["bidclose"]
        d["entry_signal"] = {d_date: kwargs.get("signal")}
        current_signal = rs_client.get("current_signal")
        current_signal_df = pd.DataFrame(d).iloc[0]
        return d
    
    def strategy1(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        last_entry_signal = self.get_last_signal()
        last_entry_signal = next(pd.DataFrame(last_entry_signal).itertuples())
        tragedy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        tragedy.dataframe = df
        tragedy.entry_signal = last_entry_signal
        tragedy.is_valid_dataframe()
        last_exit_signal= tragedy.get_signal()
        # ic(last_exit_signal)
        # ic(last_entry_signal.Index, last_entry_signal.bidclose)
        def get_trend(df):
            last_row_price = df.iloc[0].bidclose
            avr = round(df.iloc[:6].bidclose.sum() / 6, 5)
            if last_row_price > avr:
                return "UP"
            elif last_row_price < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        sma_crossover_counter = 0
        try:
            last_crossover = new_df.loc[(new_df["sma_signal_1"] == "UP_CROSSOVER") | (new_df["sma_signal_1"] == "DOWN_CROSSOVER")].iloc[-0]
        except:
            last_crossover = pd.DataFrame()
        for row in new_df.itertuples():
            if row.sma_signal_1.endswith("CROSSOVER"):
                break
            else:
                sma_crossover_counter += 1
                
        if not data.empty and signal == last_entry_signal.entry_signal:
            if "SLPR" in last_exit_signal:
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1) < 13:
                    return None
                elif not last_crossover.empty:
                    if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                        return None
                    elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                        return None
            elif last_exit_signal == (0,0,0):
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_entry_signal.Index)) / pd.Timedelta(minutes=1) < 13:
                    return None

        if last_exit_signal[2] == "TG" and not data.empty:
            entry_diff = (pd.Timestamp(data.iloc[0].name) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if signal != last_entry_signal.entry_signal and entry_diff <= 15:
                ic(last_exit_signal)
                ic(last_entry_signal)
                return self.create_signal(data, signal=last_entry_signal.entry_signal)
        
        
        if not last_crossover.empty:
            if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                return None
            elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                return None
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
       
    
    def get_previous_signal(self):
        negative_looper = list(range(-9,-1))[::-1]
        first_num = negative_looper[0]
        for n in negative_looper[1:]:
            signal = self.get_current_signal(last_signal_value=first_num, second_last_signal_value=n)
            if signal:
                return signal
            first_num = n
        return None

    
    def get_current_signal(self, **kwargs):
        """optimization required"""
        df = kwargs.get("dataframe", self.dataframe)
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("last_signal_value", -1)]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("second_last_signal_value", -2)]
        new_df = df[:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
    
    def strategy(self, **kwargs):
        """optimization required"""
        current_signal = self.get_current_signal()
        last_signal = self.get_previous_signal()
        return current_signal
        

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
    
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        


class Level3(BaseStrategy):
        
    def strategy(self):
        df1 = self.dataframe.copy()
        df2 = self.dataframe.copy()
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        strategy.dataframe = df1
        strategy.is_valid_dataframe()
        current_signal = strategy.get_current_signal()
        previous_signal = strategy.get_previous_signal()
        exit_strategy = SMAStrategy3Exit(stoplos=self.stoploss, target=self.target)
        exit_strategy.dataframe = df2
        exit_strategy.entry_signal = next(pd.DataFrame(previous_signal).itertuples())
        exit_strategy.is_valid_dataframe()
        last_exit_signal= exit_strategy.get_signal()
        # print(last_exit_signal, "exit signal for last entry siignal")
        # print(current_signal, "current signal")
        # print(previous_signal, "last signal")
        try:
            current_signal_df = next(pd.DataFrame(current_signal).itertuples())
        except:
            current_signal_df = pd.DataFrame()
            
        try:
            last_signal_df = next(pd.DataFrame(previous_signal).itertuples())
        except:
            last_signal_df = pd.DataFrame()
            
        if last_exit_signal and last_exit_signal[2] == "TG" and current_signal:
            entry_diff = (pd.Timestamp(current_signal_df.Index) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if current_signal_df.entry_signal != last_signal_df.entry_signal and entry_diff <= 15:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
                return current_signal
        if last_exit_signal == (0,0,0) and current_signal:
            last_entry_signal_price = last_signal_df.bidclose
            current_entry_signal_price = current_signal_df.bidclose
            diff = 0
            if last_signal_df.entry_signal == "BUY":
                diff = current_entry_signal_price - last_entry_signal_price
            else:
                diff = last_entry_signal_price - current_entry_signal_price
            
            if diff <= -0.0003:
                return None
            elif diff >= 0.0003:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
        return current_signal
    
    
# def custom_strategy_on_SMAStrategy3(df, stoploss, target):
#     strategy = SMAStrategy3(stoploss=stoploss, target=target)
#     ic(df.iloc[-1].name)
#     strategy.dataframe = df
#     strategy.is_valid_dataframe()
#     current_signal = strategy.get_current_signal()
#     last_signal = strategy.get_last_signal()
#     last_exit_signal = strategy.get_exit_signal(last_signal)
#     sig = pd.DataFrame(current_signal)
#     if last_exit_signal and last_exit_signal[2] == "TG" and current_signal:
#         current_signal_df = next(pd.DataFrame(current_signal).itertuples())
#         last_signal_df = next(pd.DataFrame(last_signal).itertuples())
#         entry_diff = (pd.Timestamp(current_signal_df.Index) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
#         if current_signal_df.entry_signal != last_signal_df.entry_signal and entry_diff <= 15:
#             return strategy.create_signal(sig, signal=last_signal_df.entry_signal)
#     return strategy.create_signal(sig, signal=next(sig.itertuples()).entry_signal)
    
        #     ic(last_exit_signal)
        #     ic(last_entry_signal)
        #     return self.create_signal(data, signal=last_entry_signal.entry_signal)
    # ic(current_signal)
    # ic(last_signal)
    # ic(last_exit_signal)
    
# do = custom_strategy_on_SMAStrategy3(kada, 0.0005, 0.0010)

# st = Level3(stoploss=0.0010, target=0.0005)
# test_data = data[:826]
# st.dataframe = test_data
# ic(test_data.iloc[-1])
# st.is_valid_dataframe()
# d = st.get_signal()
# d


In [33]:
class SMAStrategy2Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to7754w_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        # if signal and not data.empty:
        #     d_date = data.index[0]
        #     d = data.to_dict()
        #     d["entry_price"] = d["bidclose"]
        #     d["entry_signal"] = {d_date: signal}
        #     return d
        return df
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1)
# dd

In [ ]:
test_data = data.copy() #[15000:]
import backtest
import strategy
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)

# exit_strategy=SMAExitLevel1,
sma_backtest = BackTestStrategy(Level3, test_data, target=0.0008,stoploss=0.0005, order_quantity=30000)
sma_backtest.start_backtesting()

  0%|          | 0/25889 [00:00<?, ?it/s]

In [ ]:
import strategy
importlib.reload(strategy)
import backtest
importlib.reload(backtest)
# kd = sma_backtest.get_report()
kd = sma_backtest.clean_report(dp)
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
print(len(kd[kd["status"] == 0].pl))
print("False Signals")

In [38]:
# 1 Total Stoploss
# 22 Total Target
# 102 Total TGP
# 0 Total SLP
# 43 Total SLPR
# 1 Total Stoploss
# 124 Total Target
# Total Amount Earned 1339.7999999999379
# From Date: 2021-12-01 01:14:00
# To Date: 2021-12-03 18:42:00
# (204, 14)
# ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
# ic| kd[kd["status"] == "SLPR"].pl.sum(): -488.40000000002885
# ic| kd[kd["status"] == "TGP"].pl.sum(): 1152.8999999999724
# ic| kd[kd["status"] == "SL"].pl.sum(): -16.80000000000348
# ic| kd[kd["status"] == "TG"].pl.sum(): 692.0999999999977
# False Signals
# 36
# kd.to_csv("dec_test_half.csv")
kd.drop(["max_bid","min_bid","sma_signal"],axis=1)
# diff = []
# for k in kd.itertuples():
#     try:
#         diff.append(k.exit_time - k.Index)
#     except:
#         diff.append(0)
# kd["diff"]= diff
# kd.drop(["max_bid","min_bid"],axis=1).head(50)
# kd[kd["status"] == "SLPR"].head(60).drop(["max_bid","min_bid","small_sma_8","large_sma_16","sma_signal_1","sma_signal"],axis=1)


,bidopen,rsi,entry_price,entry_signal,exit_price,exit_time,status,pl
2021-10-15 15:53:00,1.15950,44.617921,1.15952,BUY,0.00000,0,0,0.0
2021-10-15 16:08:00,1.15936,55.330781,1.15942,BUY,1.15975,2021-10-15 17:36:00,TGP,9.9
2021-10-15 19:06:00,1.16036,57.726095,1.16036,SELL,1.16000,2021-10-15 19:32:00,TGP,10.8
2021-10-15 20:37:00,1.16005,53.843446,1.16014,BUY,0.00000,0,0,0.0
2021-10-17 21:14:00,1.15965,55.455909,1.15965,SELL,1.15950,2021-10-17 21:39:00,TGP,4.5
...,...,...,...,...,...,...,...,...
2021-10-29 18:34:00,1.15581,57.015798,1.15582,BUY,1.15611,2021-10-29 18:53:00,TGP,8.7
2021-10-29 18:58:00,1.15592,41.278191,1.15586,SELL,1.15532,2021-10-29 19:26:00,TGP,16.2
2021-10-29 19:29:00,1.15541,52.054973,1.15547,BUY,1.15582,2021-10-29 19:48:00,TGP,10.5
2021-10-29 20:10:00,1.15604,58.094101,1.15605,SELL,1.15584,2021-10-29 20:39:00,TGP,6.3


In [38]:
k = pd.Timestamp(test_signal.Index) - pd.Timedelta(250, "m")
# data[str(k):test_signal.Index]
k < pd.Timestamp(test_signal.Index)

True

In [9]:
kd = sma_backtest.clean_report(dp)[["bidopen","bidclose","entry_price","entry_signal","exit_price","exit_time","status","pl"]]
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
# kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
len(kd[kd["status"] == 0].pl)

279 Total Stoploss
57 Total Target
586 Total TGP
0 Total SLP
180 Total SLPR
279 Total Stoploss
643 Total Target
Total Amount Earned 1464.9000000000378
From Date: 2021-10-01 01:28:00
To Date: 2021-10-29 20:57:00
(1105, 8)


ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| kd[kd["status"] == "SLPR"].pl.sum(): -1653.9000000000037
ic| kd[kd["status"] == "TGP"].pl.sum(): 5869.500000000076
ic| kd[kd["status"] == "SL"].pl.sum(): -4746.600000000021
ic| kd[kd["status"] == "TG"].pl.sum(): 1995.899999999986


False Signals


3

In [11]:
dp[["exit_time", "entry_signal", "pl","entry_price","exit_price","status"]].to_csv("oct_filter_str.csv")

# Backtesting Packages

In [160]:
from tqdm.notebook import trange, tqdm
import time

# Backtesting Code

## Settings

In [161]:
entryStrategy = None
exitStrategy = None
stoploss = None
target = None
quantity = 10000
backtesting_dataframe = pd.DataFrame()

#### Start Backtesting Code

In [171]:
# entryStrategy = entryStrategy(stoploss=stoploss, target=target)

# for row_number in trange(1,len(backtesting_dataframe)):
    